<a href="https://colab.research.google.com/github/DrFrank25/Syndecan_4-Ag73/blob/main/MMPBSA_ONE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Binding Free Energy Calculation (MM-PBSA + Entropy)**

In [ ]:
# ==========================================
# 🚀 Binding Free Energy Calculation (MM-PBSA + Entropy) in Colab
# ==========================================
# -- Import Libraris ----
import os, shutil, subprocess
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# ---------- Helpers ----------
def run(cmd, **kw):
    print(f"\n$ {cmd}")
    return subprocess.run(cmd, shell=True, check=True, text=True, **kw)

In [ ]:
# ---------- 0) Micromamba + install deps ----------
if not shutil.which("micromamba"):
    run("wget -qO- https://micro.mamba.pm/api/micromamba/linux-64/latest | tar -xvj bin/micromamba")
    os.environ["MAMBA_ROOT_PREFIX"] = "/root/micromamba"
    run("./bin/micromamba shell init -s bash -p /root/micromamba")
    os.environ["PATH"] = "/root/micromamba/bin:" + os.environ["PATH"]

run("micromamba create -y -n gmx -c conda-forge gromacs gmx-mmpbsa python=3.11")
os.environ["PATH"] = "/root/micromamba/envs/gmx/bin:" + os.environ["PATH"]

MMPBSA_BIN = "gmx_MMPBSA" if shutil.which("gmx_MMPBSA") else ("g_mmpbsa" if shutil.which("g_mmpbsa") else None)
if MMPBSA_BIN is None:
    raise SystemExit("Could not find gmx_MMPBSA/g_mmpbsa.")

In [ ]:
# ---------- 1) Upload ----------
from google.colab import files
print("📂 Upload: topol.tpr, traj.xtc, index.ndx, mmpbsa.in")
uploaded = files.upload()


In [ ]:
# ---------- 2) Config QHA groups (ajuste conforme seu index.ndx) ----------
COVAR_GROUPS = "1 1"   # fit group, analysis group
ANAEIG_GROUP = "1"     # analysis group

In [ ]:
## ---------- 3) Run MM-PBSA ----------
os.makedirs("rep", exist_ok=True)
mm_out = "rep/FINAL_RESULTS_MMPBSA.dat"
run(f"{MMPBSA_BIN} -f traj.xtc -s topol.tpr -n index.ndx -i mmpbsa.in -o {mm_out} -nogui")

# Parse MM-PBSA
mm = {}
with open(mm_out) as f:
    for line in f:
        if line.strip() and not line.startswith("#"):
            parts = line.split()
            if len(parts) >= 2:
                try:
                    mm[parts[0]] = float(parts[1])
                except:
                    pass

# Normalizar chaves e calcular ΔH se necessário
key_map = {
    "DeltaE_vdw":"DeltaE_vdw","E_vdw":"DeltaE_vdw",
    "DeltaE_ele":"DeltaE_ele","E_ele":"DeltaE_ele",
    "DeltaG_pol":"DeltaG_pol","G_polar":"DeltaG_pol",
    "DeltaG_np":"DeltaG_np","G_nonpolar":"DeltaG_np",
    "DeltaTOTAL":"DeltaTOTAL","TOTAL":"DeltaTOTAL"
}
mm_norm = {key_map.get(k, k): v for k, v in mm.items()}
if "DeltaTOTAL" not in mm_norm:
    mm_norm["DeltaTOTAL"] = sum([
        mm_norm.get("DeltaE_vdw",0.0),
        mm_norm.get("DeltaE_ele",0.0),
        mm_norm.get("DeltaG_pol",0.0),
        mm_norm.get("DeltaG_np",0.0),
    ])

In [ ]:
# ---------- 4) Entropia via QHA (gmx covar + gmx anaeig) ----------
run(f"echo {COVAR_GROUPS} | gmx covar -s topol.tpr -f traj.xtc -n index.ndx -o rep/eigenval.xvg -v rep/eigenvec.trr -quiet")
run(f"echo {ANAEIG_GROUP} | gmx anaeig -v rep/eigenvec.trr -entropy rep/entropy.xvg -quiet")

TDeltaS = None
with open("rep/entropy.xvg") as f:
    for line in f:
        if line.startswith(("#","@")) or not line.strip():
            continue
        parts = line.split()
        try:
            TDeltaS = float(parts[-1])  # kJ/mol
        except:
            pass
if TDeltaS is None:
    raise RuntimeError("Could not parse -TΔS from rep/entropy.xvg")


In [ ]:
# ---------- 5) Somatórios finais ----------
row = {
    "ΔE_vdw": mm_norm.get("DeltaE_vdw", float("nan")),
    "ΔE_ele": mm_norm.get("DeltaE_ele", float("nan")),
    "ΔG_pol": mm_norm.get("DeltaG_pol", float("nan")),
    "ΔG_np":  mm_norm.get("DeltaG_np",  float("nan")),
}
row["ΔH"] = row["ΔE_vdw"] + row["ΔE_ele"] + row["ΔG_pol"] + row["ΔG_np"]
row["-TΔS"] = TDeltaS
row["ΔG_total"] = row["ΔH"] + row["-TΔS"]

df = pd.DataFrame([row], index=["Representative"])
display(df)


In [ ]:
# ---------- 6) Export ----------
df.to_csv("Binding_Energy_Representative.csv", float_format="%.4f")
print("✅ Saved: Binding_Energy_Representative.csv")

In [ ]:
# ---------- 7) Gráficos ----------
# (a) Decomposição completa (inclui ΔH, -TΔS, ΔG_total)
terms_all = ["ΔE_vdw","ΔE_ele","ΔG_pol","ΔG_np","ΔH","-TΔS","ΔG_total"]
ax = df[terms_all].T.plot(kind="bar", figsize=(10,6), edgecolor="black")
ax.set_ylabel("Energy (kJ/mol)")
ax.set_title("Energy Decomposition — Representative Replicate")
ax.set_xticklabels(terms_all, rotation=45, ha="right")
plt.tight_layout()
plt.savefig("Energy_Decomposition_Representative.png", dpi=300)
plt.show()
print("📊 Saved: Energy_Decomposition_Representative.png")

# (b) Somente contribuições energéticas solicitadas
terms_core = ["ΔE_vdw","ΔE_ele","ΔG_pol","ΔG_np"]
ax2 = df[terms_core].T.plot(kind="bar", figsize=(8,5), edgecolor="black")
ax2.set_ylabel("Energy (kJ/mol)")
ax2.set_title("Core Energy Terms — Representative Replicate")
ax2.set_xticklabels(terms_core, rotation=45, ha="right")
plt.tight_layout()
plt.savefig("Energy_CoreTerms_Representative.png", dpi=300)
plt.show()
print("📊 Saved: Energy_CoreTerms_Representative.png")

print("✅ Done.")